In [1]:
from pgsql_search.loader import HuggingFaceDatasets

ds = HuggingFaceDatasets("UCSC-VLAA/Recap-COCO-30K")
ds.dataset = ds.dataset.shuffle()
ds = ds.select(list(range(10)))
ds.save_images("../data/images10")
ds = ds.select_columns(["image_filepath", "caption"])
ds.dataset


2024-12-18 16:44:56.675 | INFO     | pgsql_search.loader:__init__:15 - Loading dataset: UCSC-VLAA/Recap-COCO-30K
2024-12-18 16:44:56.683 | INFO     | pgsql_search.loader:save_images:44 - Saving images to folder: ../data/images10


Saving images (num_proc=8):   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['image_filepath', 'caption'],
    num_rows: 10
})

In [2]:
df = ds.dataset.to_pandas()
df

,image_filepath,caption
0,../data/images10/243158.jpg,The living room has a open feel to it
1,../data/images10/77889.jpg,an image of a plate of food with drinks
2,../data/images10/500752.jpg,Two elephants in the water interacting with on...
3,../data/images10/280008.jpg,Several people that are playing in a frisbee g...
4,../data/images10/258793.jpg,a number of buildings in the distance
5,../data/images10/194555.jpg,A sign that says Church Avenue in front of a f...
6,../data/images10/295269.jpg,a bunch of stacked cookies on a table
7,../data/images10/296564.jpg,Someone in a blue shirt and calf-high pants is...
8,../data/images10/138503.jpg,A man in a uniform and helmet rides a motorcyc...
9,../data/images10/289813.jpg,a young boy sits in the grass outside


In [3]:
from pgsql_search.database import PostgreSQLDatabase, ColumnType

In [4]:
with PostgreSQLDatabase("retrieval_db") as db:
    # First, create the table with just an ID column
    db.initialize_table("image_metadata")
    db.add_column("image_filepath", ColumnType.TEXT, nullable=False)
    db.add_column("caption", ColumnType.TEXT, nullable=True)


    db.insert_dataframe(df)

2024-12-18 16:44:57.135 | INFO     | pgsql_search.database:connect:87 - Connected to database
2024-12-18 16:44:57.142 | INFO     | pgsql_search.database:setup_pgvector_extension:104 - pgvector extension initialized
2024-12-18 16:44:57.148 | INFO     | pgsql_search.database:initialize_table:126 - Initialized table 'image_metadata' with ID column 'id'
2024-12-18 16:44:57.151 | INFO     | pgsql_search.database:add_columns:175 - Added 1 new columns to image_metadata
2024-12-18 16:44:57.153 | INFO     | pgsql_search.database:add_columns:175 - Added 1 new columns to image_metadata
2024-12-18 16:44:57.161 | INFO     | pgsql_search.database:insert_dataframe:227 - Inserted batch 1 (10/10 rows)
2024-12-18 16:44:57.162 | INFO     | pgsql_search.database:insert_dataframe:231 - Successfully inserted 10 rows into image_metadata
2024-12-18 16:44:57.163 | INFO     | pgsql_search.database:disconnect:96 - Disconnected from database
